In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print('', end = '')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import keras
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import Flatten, Lambda
from tensorflow.nn import depth_to_space

In [4]:
def discriminator_model(in_shape = (2,256,256,1)):
    model = Sequential()
    
    args = {
        'activation': 'relu',
        'kernel_initializer': 'Orthogonal',
        'padding': 'same',
        'kernel_size': (3,3),
    }
    
    model.add(Input(shape = in_shape))
    model.add(Conv2D(filters = 64, **args))
    model.add(Conv2D(filters = 64, strides = 2, **args))
    model.add(Conv2D(filters = 128, **args))
    model.add(Conv2D(filters = 128, strides = 2, **args))
    model.add(Conv2D(filters = 256, **args))
    model.add(Conv2D(filters = 256, strides = 2, **args))
    model.add(Conv2D(filters = 512, **args))
    model.add(Conv2D(filters = 512, strides = 2, **args))
    model.add(Conv2D(filters = 512, **args))
    model.add(Conv2D(filters = 512, strides = 2, **args))
    model.add(Flatten())
    model.add(Dense(2048, activation = 'relu'))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(2, activation = 'sigmoid'))
    
    return model

In [5]:
dis_mod = discriminator_model()
dis_mod.summary()

In [6]:
def generator_model(in_shape = (256, 256, 1)):
    model = Sequential()
    
    model.add(Input(shape = in_shape))
    
    args = {
        'activation':'relu',
        'kernel_initializer':'Orthogonal'
    }
    
    model.add(Conv2D(filters = 16, kernel_size = (5,5), padding = 'same', strides = 2, **args))
    model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'same', strides = 2, **args))
    model.add(Conv2D(filters = 128, kernel_size = (5,5), padding = 'same', strides = 2, **args))
    model.add(Conv2D(filters = 512, kernel_size = (5,5), padding = 'same', strides = 2, **args))
    model.add(Conv2D(filters = 1024, kernel_size = (5,5), padding = 'same', strides = 2, **args))
    
    model.add(Conv2D(filters = 512*4, kernel_size = (3,3), padding = 'same', strides = 1, **args))
    model.add(Lambda(lambda x: depth_to_space(x, 2)))
    model.add(Conv2D(filters = 128*4, kernel_size = (3,3), padding = 'same', strides = 1, **args))
    model.add(Lambda(lambda x: depth_to_space(x, 2)))
    model.add(Conv2D(filters = 64*4, kernel_size = (3,3), padding = 'same', strides = 1, **args))
    model.add(Lambda(lambda x: depth_to_space(x, 2)))
    model.add(Conv2D(filters = 16*4, kernel_size = (3,3), padding = 'same', strides = 1, **args))
    model.add(Lambda(lambda x: depth_to_space(x, 2)))
    model.add(Conv2D(filters = 1*4, kernel_size = (3,3), padding = 'same', strides = 1, **args))
    model.add(Lambda(lambda x: depth_to_space(x, 2)))
    
    return model

In [7]:
gen_mod = generator_model()
gen_mod.summary()

In [8]:
def generator_loss(alpha, dis_mod):
    
    def find_loss(y_true, y_pred):
        mse = (np.linalg.norm(y_true[0] - y_pred))**2
        target = y_true[1]
        return (-np.log(dis_mod.predict(np.array([y_pred, target]))) + alpha*mse)
    
    return find_loss

In [9]:
def discriminator_loss(y_true, y_pred):
    return np.log(y_pred) - np.log(y_true)

In [10]:
def training_gan(gen_mod, dis_mod, targets, masks, num_epochs = 100, alpha = 0.005, lr = 0.01, batch_size = 100):
    tot_len = len(targets)
    for i in range(num_epochs):
        mask_batch = []
        target_batch = []
        gen_masks = []
        dis_inp = []
        dis_vals = []
        for j in range(batch_size):
            new_mask = gen_mod.predict(targets[(i*batch_size + j)%tot_len])
            dis_inp.append(np.array([new_mask, targets[(i*batch_size + j)%tot_len]]))
            
            dis_gnd_mask = np.array([masks[(i*batch_size + j)%tot_len], targets[(i*batch_size + j)%tot_len]])
            dis_vals.append(dis_mod.predict(dis_gnd_mask))
            
            mask_batch.append(dis_gnd_mask)
            target_batch.append(targets[(i*batch_size + j)%tot_len])
        
        gen_mod.compile(loss = generator_loss(alpha, dis_mod), optimizer = Adam(lr=lr))
        dis_mod.compile(loss = discriminator_loss, optimizer = Adam(lr=lr))
        
        gen_mod.fit(target_batch, mask_batch, epochs = 1, batch_size = batch_size)
        dis_mod.fit(dis_inp, dis_vals, epochs = 1, batch_size = batch_size)
    return gen_mod

In [12]:
import cv2
import glob

In [24]:
target_files = glob.glob('../input/gan-opc-dataset-2/ganopc-data/artitgt/*.png')
mask_files = glob.glob('../input/gan-opc-dataset-2/ganopc-data/artimsk/*.png')

target_files.sort()
mask_files.sort()

In [ ]:
targets = [cv2.imread(target_file) for target_file in target_files]
masks = [cv2.imread(mask_file) for mask_file in mask_files]

In [ ]:
training_gan(gen_mod, dis_mod, targets, masks)